In [1]:
# Import packages
library(rstan)
library(coda)
library(parallel)
library(Bessel)

Loading required package: StanHeaders
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
rstan (Version 2.19.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
For improved execution time, we recommend calling
Sys.setenv(LOCAL_CPPFLAGS = '-march=native')
although this causes Stan to throw an error on a few processors.

Attaching package: 'coda'

The following object is masked from 'package:rstan':

    traceplot



In [2]:
# add path required by rstan
library(devtools) 
Sys.setenv(PATH = paste("C:/Rtools/bin", Sys.getenv("PATH"), sep=";")) 
Sys.setenv(PATH = paste("C:\\Rtools\\mingw_64\\bin", Sys.getenv("PATH"), sep=";")) 
Sys.setenv(BINPREF = "C:/Rtools/mingw_$(WIN)/bin/")

Loading required package: usethis


# Read csv into dataframe

In [3]:
dat.str <- Sys.glob("data.*.csv")
data <- as.data.frame(read.csv(file = dat.str))
# slicing training set (whole 2018 season)
dat <- data[data$period < 38,]

In [4]:
head(dat)

,X,Date,Season,round,period,home.name,away.name,home.score,away.score,home.goals,...,Total.Score.Over.Min,Total.Score.Over.Max,Total.Score.Over.Close,Total.Score.Under.Open,Total.Score.Under.Min,Total.Score.Under.Max,Total.Score.Under.Close,Notes,home.number,away.number
64,82,2018-06-24,2018,14,37,CW,CA,79,59,11,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,16,10
65,83,2018-06-23,2018,14,37,WB,NM,75,77,11,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,17,15
66,84,2018-06-23,2018,14,37,BL,GW,82,109,12,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,11,7
67,85,2018-06-23,2018,14,37,HW,GC,96,43,13,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,14,5
68,86,2018-06-22,2018,14,37,PA,ME,75,65,11,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,1,6
69,87,2018-06-21,2018,14,37,WC,ES,52,80,6,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,4,2


In [5]:
# library(hash)

# h <- hash()

# for (i in 1:length(sort(unique(dat$home.name)))){
#     h[[as.character(sort(unique(dat$home.name))[i])]] <- i
# }

In [82]:
# library(hash)

# h <- hash()

# for (i in 1:length(sort(unique(dat$home.name)))){
#     h[[as.character(sort(unique(dat$home.name))[i])]] <- i
# }

# home.number <- vector()
# away.number <- vector()

# for (i in 1:nrow(dat)){
#     home.number <- append(home.number, h[[as.character(dat$home.name[i])]])
#     away.number <- append(away.number, h[[as.character(dat$away.name[i])]])
    
# }

# dat$home.number <- home.number
# dat$away.number <- away.number

In [84]:
# write.csv(dat, 'temp.csv')

# 2019 Round 1 data

In [5]:
matchup <- data[data$period == 38,]
n.matches <- length(matchup$home.number)
matchup

,X,Date,Season,round,period,home.name,away.name,home.score,away.score,home.goals,...,Total.Score.Over.Min,Total.Score.Over.Max,Total.Score.Over.Close,Total.Score.Under.Open,Total.Score.Under.Min,Total.Score.Under.Max,Total.Score.Under.Close,Notes,home.number,away.number
55,73,2018-07-01,2018,15,38,FR,BL,64,119,9,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,8,17
56,74,2018-07-01,2018,15,38,ES,NM,125,108,19,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,15,16
57,75,2018-07-01,2018,15,38,ME,SK,117,119,18,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,9,14
58,76,2018-06-30,2018,15,38,GC,CW,60,99,8,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,4,10
59,77,2018-06-30,2018,15,38,GW,HW,95,84,13,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,2,1
60,78,2018-06-30,2018,15,38,AD,WC,88,78,12,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,5,18
61,79,2018-06-30,2018,15,38,CA,PA,69,90,10,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,11,12
62,80,2018-06-29,2018,15,38,WB,GE,103,101,16,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,6,3
63,81,2018-06-28,2018,15,38,RI,SY,93,67,14,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,,8,13


# Data

In [6]:
home.name <- as.matrix(dat$home.name)
away.name <- as.matrix(dat$away.name)
home.numb <- dat$home.number
away.numb <- dat$away.number

home.Goals <- dat$home.goals
home.Behinds <- dat$home.behinds
away.Goals <- dat$away.goals
away.Behinds <- dat$away.behinds

period <- dat$period
n.teams <- 18
n.games <- length(home.numb)
n.periods <- max(period)
n.bps <- 2
bps <- c(1,24)

In [7]:
# matricies for the Owen paper centering
ones <- matrix(1, nrow = (n.teams-1), ncol = 1)
S_t <-  n.teams / (n.teams-1) * (diag(n.teams-1) + ones %*% t(ones))

# Jmat is n * n-1
J.up.left <- diag(n.teams-1) - (1/n.teams)*(ones %*% t(ones))
J.bot.left <- - t(ones) * 1/n.teams

Jmat <- rbind(J.up.left, J.bot.left)

# Store data into list required by STAN

In [8]:
dat.list <- list(N = n.teams,
                 G = n.games,
                 T = n.periods,
                 period = period,
                 home = home.numb,
                 away = away.numb,
                 homeGoals = home.Goals,
                 homeBehinds = home.Behinds,
                 awayGoals = away.Goals,
                 awayBehinds = away.Behinds,
                 BP = n.bps,
                 Breaks = bps,
                 Smat = S_t,
                 JmatTrans = t(Jmat),
                 N_Matches = n.matches,
                 nextHome = matchup$home.number,
                 nextAway = matchup$away.number)

# Compile the model

In [9]:
prefit <- stan("Skellam goal and behind difference.stan", data = dat.list, chains = 0)

DIAGNOSTIC(S) FROM PARSER:
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # are deprecated.  Please use // in place of # for line comments.
Info: Comments beginning with # 

the number of chains is less than 1; sampling not done


# Sampling using MCMC (4 cores in parallel)

In [10]:
cl <- makeCluster(4)

clusterEvalQ(cl, library(rstan))

myfunction <- function(var1, var2, var3){
    stan(fit = var3, data = var2,
                            chains = 1, chain_id = var1, iter = 2500,
                            refresh = -1, init = "0")
}

sflist <- parLapply(cl, 1:4, fun = myfunction, var2 = dat.list, var3 = prefit)
stopCluster(cl)

[[1]]
 [1] "rstan"       "ggplot2"     "StanHeaders" "stats"       "graphics"   
 [6] "grDevices"   "utils"       "datasets"    "methods"     "base"       

[[2]]
 [1] "rstan"       "ggplot2"     "StanHeaders" "stats"       "graphics"   
 [6] "grDevices"   "utils"       "datasets"    "methods"     "base"       

[[3]]
 [1] "rstan"       "ggplot2"     "StanHeaders" "stats"       "graphics"   
 [6] "grDevices"   "utils"       "datasets"    "methods"     "base"       

[[4]]
 [1] "rstan"       "ggplot2"     "StanHeaders" "stats"       "graphics"   
 [6] "grDevices"   "utils"       "datasets"    "methods"     "base"

In [11]:
sflist

[[1]]
Inference for Stan model: Skellam goal and behind difference.
1 chains, each with iter=2500; warmup=1250; thin=1; 
post-warmup draws per chain=1250, total post-warmup draws=1250.

                           mean se_mean    sd    2.5%    25%    50%    75%
delta                      0.70    0.00  0.06    0.58   0.66   0.70   0.74
attackGoals_raw[1,1]       1.83    0.13  2.91   -4.15  -0.11   1.91   3.92
attackGoals_raw[1,2]      -4.40    0.12  3.07   -8.87  -6.84  -4.81  -2.41
attackGoals_raw[1,3]      -2.97    0.13  3.60   -8.77  -5.68  -3.18  -0.60
attackGoals_raw[1,4]      -0.33    0.13  3.30   -6.86  -2.55  -0.27   1.88
attackGoals_raw[1,5]       0.62    0.15  2.80   -4.79  -1.25   0.58   2.52
attackGoals_raw[1,6]       1.01    0.13  3.15   -5.04  -1.16   1.06   3.20
attackGoals_raw[1,7]      -0.06    0.12  2.69   -5.23  -1.86  -0.04   1.73
attackGoals_raw[1,8]       1.08    0.15  3.07   -4.83  -1.19   0.95   3.30
attackGoals_raw[1,9]       1.24    0.09  2.00   -2.53  -0.18   1

In [11]:
fit <- sflist2stanfit(sflist)

In [12]:
write.csv(print(summary(fit)), file = "fitproper.csv")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Extract parameters

In [13]:
fit.samples <- extract(fit)
write.csv(fit.samples, file = "samples.csv")

# Take the average of the mu for each team

In [14]:
# parameter estimate extractor
delta <- mean (fit.samples$delta)
muAttGoals.vec <- apply(fit.samples$muAttGoals, 2, mean) # I think the 18th team here is
  # suffering from the sum to zero constraint, maybe will dissapear with more samples

muAttBehinds.vec <- apply(fit.samples$muAttBehinds, 2, mean)
muDef.vec <- apply(fit.samples$muDef, 2, mean)

mu.ests <- matrix(c( "Goal attack mean estimates", muAttGoals.vec,"Behind attack mean estimates", muAttBehinds.vec,
                     "defend means estimates", muDef.vec), ncol = 3, byrow = F)
write.csv(mu.ests, file = "AR mean estimates.csv")

# Extract phiAtt, phiDef, attackGoals, attackBehinds, defend

In [15]:
phiAtt.vec <- mean(fit.samples$phiAtt)
phiDef.vec <- mean(fit.samples$phiDef)
std.attack.vec <- mean(fit.samples$stdAtt)
std.def.vec <- mean(fit.samples$stdDef)

goal.attack.strenghts.mat <- apply(fit.samples$attackGoals, 2:3, mean)
write.csv(goal.attack.strenghts.mat, file = "goal.attack.str.csv")

behind.attack.strengths.mat <- apply(fit.samples$attackBehinds, 2:3, mean)
write.csv(behind.attack.strengths.mat, file = "behind.attack.str.csv")

defence.strengths.mat <- apply(fit.samples$defend, 2:3, mean)
write.csv(defence.strengths.mat, file = "def.str.csv")

par.vec <- c(delta, "delta",
             phiAtt.vec, "phi.attack",
             phiDef.vec, "phi.defend",
             std.attack.vec, "std.attack",
             std.def.vec, "std.defend")

par.vec.est <- matrix(par.vec, ncol = 2, byrow = T)
write.csv(par.vec.est,  file = "parameter estimates.csv")

# Extract goalDiff, behindDiff, and totalDiff

In [16]:
avg.pred.goal.diff <- apply(fit.samples$goalDiff, 2, mean)
avg.pred.behind.diff <- apply(fit.samples$behindDiff, 2, mean)
avg.pred.total.diff <- apply(fit.samples$totalDiff, 2, mean)

In [17]:
# see if we got it right
actual.total.diff <- matchup$home.score - matchup$away.score
correct.vec <- ((avg.pred.total.diff * actual.total.diff)>0)

In [18]:
actual.total.diff
correct.vec

[1] -55  17  -2 -39  11  10 -21   2  26

[1]  TRUE  TRUE FALSE FALSE  TRUE FALSE  TRUE  TRUE  TRUE

# Calculate percentage of win, loss and draw

In [19]:
# get the percentage of win/loss

winDrawLossPer <- function(x.vec) {
  home.win.per <- mean(as.numeric(x.vec > 0))
  draw.per <- mean(as.numeric(x.vec == 0))
  away.win.per <- mean(as.numeric(x.vec < 0))
  c(home.win.per, draw.per ,away.win.per)
}

In [20]:
per.chance.mat <- apply(fit.samples$totalDiff, 2, winDrawLossPer)
per.chance.mat

0.4028,0.7440,0.5288,0.6666,0.8420,0.3176,0.3882,0.8306,0.5356
0.0314,0.0354,0.0352,0.0182,0.0156,0.0342,0.0312,0.0514,0.0416
0.5658,0.2206,0.4360,0.3152,0.1424,0.6482,0.5806,0.1180,0.4228


In [21]:
# the sd ests are going to be wildly larger than they should due to not throwing out burn in
# samples, should probably change all my apply's to deal with this but ehhhhhhhh, more samples
# takes care of it for the most part
sd.ests <- round(apply(fit.samples$totalDiff,2,sd),2)

In [22]:
sim.pred.mat <- matrix(c("behind diff", avg.pred.behind.diff,
                         "goal diff", avg.pred.goal.diff,
                         "total diff", avg.pred.total.diff,
                         "home team", as.character(matchup$home.name),
                         "away team", as.character(matchup$away.name),
                         "actual result", actual.total.diff,
                         "prediction correct?", correct.vec,
                         "home win prob", per.chance.mat[1,],
                         "draw prob", per.chance.mat[2,],
                         "home loss prob", per.chance.mat[3,],
                         "simulated sd ests", sd.ests
                         ), nrow = 11, byrow = T)

write.csv(sim.pred.mat, file = "internal predictions.csv")

In [23]:
sim.pred.mat

behind diff,-3.9416,-0.8122,0.6794,11.0526,2.1124,-2.3302,-0.0166,7.746,1.6518
goal diff,-0.2386,5.4546,1.6156,0.3286,14.3052,-1.791,-1.5012,3.596,0.8326
total diff,-5.3732,31.9154,10.373,13.0242,87.9436,-13.0762,-9.0238,29.322,6.6474
home team,FR,ES,ME,GC,GW,AD,CA,WB,RI
away team,BL,NM,SK,CW,HW,WC,PA,GE,SY
actual result,-55,17,-2,-39,11,10,-21,2,26
prediction correct?,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,TRUE,TRUE
home win prob,0.4028,0.744,0.5288,0.6666,0.842,0.3176,0.3882,0.8306,0.5356
draw prob,0.0314,0.0354,0.0352,0.0182,0.0156,0.0342,0.0312,0.0514,0.0416
home loss prob,0.5658,0.2206,0.436,0.3152,0.1424,0.6482,0.5806,0.118,0.4228
simulated sd ests,48.31,74,73.76,43.57,154.35,54.71,70.93,55.66,57.72


In [24]:
write.csv(t(sim.pred.mat),  file = "round15_prediction.csv")